#### Fetching the weather data using `Open-Meteo API`

In [12]:
import requests
import pandas as pd

# Fetch daily historical weather for a single station using Open-Meteo API
def fetch_station_weather(lat, lon, station_id,
                          start_date="2023-01-01",
                          end_date="2023-12-31",
                          timezone="UTC"):
    url = "https://archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum",
        "timezone": timezone,
    }

    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json()

    daily = data.get("daily", {})
    if not daily:
        raise ValueError(f"No 'daily' data returned for station {station_id}")

    df = pd.DataFrame(daily)
    # Rename for convenience
    df = df.rename(columns={
        "time": "date",
        "temperature_2m_max": "TMAX",
        "temperature_2m_min": "TMIN",
        "temperature_2m_mean": "TAVG",
        "precipitation_sum": "PRCP",
    })

    df["InfrastructureID"] = station_id

    return df


In [14]:
# Import the EV dataset
ev_stations_df = pd.read_csv("EV stations data.csv")

# Keep only needed columns & drop missing coords
stations = ev_stations_df[["InfrastructureID", "Latitude", "Longitude"]].dropna()

all_weather_frames = []

# Fetching data
for idx, row in stations.iterrows():
    station_id = row["InfrastructureID"]
    lat = row["Latitude"]
    lon = row["Longitude"]

    try:
        print(f"Fetching weather for station {station_id} (lat={lat}, lon={lon})...")
        df_weather = fetch_station_weather(
            lat=lat,
            lon=lon,
            station_id=station_id,
            start_date="2023-01-01",
            end_date="2023-12-31",
            timezone="UTC",
        )

        all_weather_frames.append(df_weather)

    except Exception as e:
        print(f"Failed for station {station_id}: {e}")

# Combine all stations' weather into one DataFrame
weather_all_stations = pd.concat(all_weather_frames, ignore_index=True)

print(weather_all_stations.head())
print(weather_all_stations.shape)

# Save the result 
weather_all_stations.to_csv("weather_by_station_2023.csv", index=False)


Fetching weather for station PSP002216 (lat=48.42459974688293, lon=-123.36109330080023)...
Fetching weather for station PSP002223 (lat=48.44698873114609, lon=-123.33436776072703)...
Fetching weather for station PSP002225 (lat=48.442372449468046, lon=-123.3312384269044)...
Fetching weather for station PSP000511 (lat=48.42283676660572, lon=-123.36771459712304)...
Fetching weather for station PSP000629 (lat=48.42949218108236, lon=-123.368619324203)...
Fetching weather for station PSP002221 (lat=48.45004550369552, lon=-123.33805136371492)...
Fetching weather for station PSP000594 (lat=48.43035017550942, lon=-123.361352345635)...
Fetching weather for station PSP002217 (lat=48.42873114973296, lon=-123.36061464230008)...
Fetching weather for station PSP002219 (lat=48.42713330649277, lon=-123.36366933183588)...
Fetching weather for station PSP000512 (lat=48.42365072711553, lon=-123.36638931887911)...
Fetching weather for station PSP002213 (lat=48.42511383587826, lon=-123.36895447268292)...
Fet